In [ ]:
import torch, torch.nn.functional as F
import matplotlib.pyplot as plt

# Input
x = torch.randn(1,5,8)

# Q K V
WQ,WK,WV = torch.randn(8,8),torch.randn(8,8),torch.randn(8,8)
Q,K,V = x@WQ, x@WK, x@WV

# Scaled Dot-Product Attention
attn = F.softmax((Q@K.transpose(-2,-1))/(8**0.5), dim=-1)
out = attn@V

print(out.shape)

# Visualize
plt.imshow(attn[0].detach().numpy())
plt.colorbar()
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.Wq = nn.Linear(d_model, d_model)
        self.Wk = nn.Linear(d_model, d_model)
        self.Wv = nn.Linear(d_model, d_model)

    def forward(self, x):
        Q = self.Wq(x)
        K = self.Wk(x)
        V = self.Wv(x)

        scores = Q @ K.transpose(-2,-1) / (x.size(-1)**0.5)
        attn = F.softmax(scores, dim=-1)
        out = attn @ V
        return out, attn

# Example
x = torch.randn(1,5,8)        # (batch, seq_len, embed_dim)
sa = SelfAttention(8)
output, weights = sa(x)

print(output.shape)


In [ ]:
import torch, torch.nn.functional as F

# input
x = torch.randn(1,5,8)
h, dk = 2, 4

# Q K V
Wq,Wk,Wv = torch.randn(8,8),torch.randn(8,8),torch.randn(8,8)
Q = (x@Wq).view(1,5,h,dk).transpose(1,2)
K = (x@Wk).view(1,5,h,dk).transpose(1,2)
V = (x@Wv).view(1,5,h,dk).transpose(1,2)

# multi-head attention
A = F.softmax((Q@K.transpose(-2,-1))/(dk**0.5), dim=-1)
heads = A@V

print("Head1:", heads[0,0])
print("Head2:", heads[0,1])


In [ ]:
# pip install transformers matplotlib

import torch, matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel

tok = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)

text = "deep learning uses attention mechanism"
inputs = tok(text, return_tensors="pt")

out = model(**inputs)
attn = out.attentions[0][0,0].detach()   # layer0, head0

plt.imshow(attn)
plt.xticks(range(len(inputs.input_ids[0])), tok.convert_ids_to_tokens(inputs.input_ids[0]), rotation=90)
plt.yticks(range(len(inputs.input_ids[0])), tok.convert_ids_to_tokens(inputs.input_ids[0]))
plt.title("Attention Map (Layer0 Head0)")
plt.colorbar()
plt.show()


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tok = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

inp = tok("attention heads test", return_tensors="pt")
base = model(**inp).logits

for h in range(model.config.num_attention_heads):
    mask = torch.ones(model.config.num_hidden_layers, model.config.num_attention_heads)
    mask[:,h] = 0
    diff = (base - model(**inp, head_mask=mask).logits).abs().mean()
    print("Head",h,"impact:",diff.item())


In [ ]:
import torch
import torch.nn as nn
import math

class EncoderBlock(nn.Module):
    def __init__(self, d_model=64, heads=4, ff=128):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, heads, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff), nn.ReLU(), nn.Linear(ff, d_model)
        )
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        a,_ = self.attn(x,x,x)     # self-attention
        x = self.norm1(x + a)
        f = self.ffn(x)            # feed-forward
        x = self.norm2(x + f)
        return x

# test
x = torch.randn(1,10,64)
enc = EncoderBlock()
print(enc(x).shape)


In [ ]:
class SinPosEnc(nn.Module):
    def __init__(self, d_model, max_len=50):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0,max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0,d_model,2)*(-math.log(10000)/d_model))
        pe[:,0::2] = torch.sin(pos*div)
        pe[:,1::2] = torch.cos(pos*div)
        self.pe = pe.unsqueeze(0)

    def forward(self,x):
        return x + self.pe[:,:x.size(1)]


In [ ]:
class LearnedPosEnc(nn.Module):
    def __init__(self, max_len=50, d_model=64):
        super().__init__()
        self.emb = nn.Embedding(max_len, d_model)

    def forward(self,x):
        pos = torch.arange(x.size(1)).unsqueeze(0)
        return x + self.emb(pos)


In [ ]:
x = torch.randn(1,10,64)

sin_pe = SinPosEnc(64)
learn_pe = LearnedPosEnc()

print("Sinusoidal:", sin_pe(x).shape)
print("Learned:", learn_pe(x).shape)


In [ ]:
import torch
import torch.nn as nn

# Short Transformer Encoder Block
class EncoderBlock(nn.Module):
    def __init__(self, d_model, heads, d_ff):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, heads, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        a,_ = self.attn(x, x, x)
        x = self.norm1(x + a)
        f = self.ffn(x)
        x = self.norm2(x + f)
        return x

